In [1]:
import sys
import urllib.parse
import urllib.request
import httplib2 as http
import json
from collections import Counter

import pandas as pd
from neo4j import GraphDatabase

import mygene

from nxontology.imports import from_file
from nxontology.viz import create_similarity_graphviz

from goatools.anno.gaf_reader import GafReader
from goatools.base import download_ncbi_associations
from goatools.anno.genetogo_reader import Gene2GoReader

# Introduction

The purpose of this notebook is to explore the types of data that can be obtain from the disease ontology knowledge graphe for the purpose of disease clustering. We obtain and explore the following types of relationships:

* Disease - Gene
* Disease - Phenotype


In [2]:
# Connect to disease ontology graph neo4j database

uri = "bolt://disease.ncats.io:80"
driver = GraphDatabase.driver(uri, auth=("neo4j", ""))

# Disease - Gene

### Obtaining GARD disease to Orphanet disease-gene mappings

In [3]:
gard_gene_query = \
"""
MATCH p=(d:DATA)-[:PAYLOAD]->(:S_GARD)-[:R_exactMatch|:R_equivalentClass]-(m:S_MONDO) WHERE d.is_rare = TRUE
OPTIONAL MATCH q=(m)-[:R_exactMatch|:R_equivalentClass]-(:S_ORDO_ORPHANET)-[e:R_rel{name:'disease_associated_with_gene'}]->(:S_ORDO_ORPHANET)<-[:PAYLOAD]-(o:DATA)
RETURN DISTINCT d.id as `GARD_ID`,d.name as `GARD_Disease`,e.DisorderGeneAssociationType as `Disease_Gene_Association`,
o.symbol as `Gene_Symbol`,o.label as `Gene_Name`, o.hasDbXref as `Gene_Refs`
ORDER BY d.id
"""

gard_gene_res = driver.session().run(gard_gene_query)

In [4]:
#Convert to dict
gard_gene_data = [dict(record) for record in gard_gene_res]

#Extract the external db references
for gd in gard_gene_data:
    if gd['Gene_Refs'] is not None:
        for ref_id in gd['Gene_Refs']:
            ref = ref_id.split(':')
            gd[ref[0]] = ref[1]

In [5]:
gard_gene_data

[{'GARD_ID': 1,
  'GARD_Disease': 'GRACILE syndrome',
  'Disease_Gene_Association': 'Disease-causing germline mutation(s) in',
  'Gene_Symbol': 'BCS1L',
  'Gene_Name': 'BCS1 homolog, ubiquinol-cytochrome c reductase complex chaperone',
  'Gene_Refs': ['OMIM:603647',
   'Ensembl:ENSG00000074582',
   'Genatlas:BCS1L',
   'SwissProt:Q9Y276',
   'HGNC:1020',
   'Reactome:Q9Y276'],
  'OMIM': '603647',
  'Ensembl': 'ENSG00000074582',
  'Genatlas': 'BCS1L',
  'SwissProt': 'Q9Y276',
  'HGNC': '1020',
  'Reactome': 'Q9Y276'},
 {'GARD_ID': 3,
  'GARD_Disease': 'Ablepharon macrostomia syndrome',
  'Disease_Gene_Association': 'Disease-causing germline mutation(s) in',
  'Gene_Symbol': 'TWIST2',
  'Gene_Name': 'twist family bHLH transcription factor 2',
  'Gene_Refs': ['Ensembl:ENSG00000233608',
   'OMIM:607556',
   'Reactome:Q8WVJ9',
   'Genatlas:TWIST2',
   'SwissProt:Q8WVJ9',
   'HGNC:20670'],
  'Ensembl': 'ENSG00000233608',
  'OMIM': '607556',
  'Reactome': 'Q8WVJ9',
  'Genatlas': 'TWIST2',
  '

In [6]:
gard_gene_df = pd.DataFrame(gard_gene_data)
gard_gene_df

,GARD_ID,GARD_Disease,Disease_Gene_Association,Gene_Symbol,Gene_Name,Gene_Refs,OMIM,Ensembl,Genatlas,SwissProt,HGNC,Reactome,IUPHAR
0,1,GRACILE syndrome,Disease-causing germline mutation(s) in,BCS1L,"BCS1 homolog, ubiquinol-cytochrome c reductase...","[OMIM:603647, Ensembl:ENSG00000074582, Genatla...",603647,ENSG00000074582,BCS1L,Q9Y276,1020,Q9Y276,NaN
1,3,Ablepharon macrostomia syndrome,Disease-causing germline mutation(s) in,TWIST2,twist family bHLH transcription factor 2,"[Ensembl:ENSG00000233608, OMIM:607556, Reactom...",607556,ENSG00000233608,TWIST2,Q8WVJ9,20670,Q8WVJ9,NaN
2,4,Acanthocheilonemiasis,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,Abetalipoproteinemia,Disease-causing germline mutation(s) in,MTTP,microsomal triglyceride transfer protein,"[Ensembl:ENSG00000138823, Genatlas:MTTP, React...",157147,ENSG00000138823,MTTP,P55157,7467,P55157,NaN
4,6,Acromesomelic dysplasia,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7790,13390,12q14 microdeletion syndrome,Role in the phenotype of,LEMD3,LEM domain containing 3,"[Reactome:Q9Y2U8, SwissProt:Q9Y2U8, OMIM:60784...",607844,ENSG00000174106,LEMD3,Q9Y2U8,28887,Q9Y2U8,NaN
7791,13390,12q14 microdeletion syndrome,Role in the phenotype of,HMGA2,high mobility group AT-hook 2,"[SwissProt:P52926, Reactome:P52926, HGNC:5009,...",600698,ENSG00000149948,HMGA2,P52926,5009,P52926,NaN
7792,13391,2p15p16.1 microdeletion syndrome,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7793,13392,16p13.11 microduplication syndrome,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
len(set(gard_gene_df[(gard_gene_df['Gene_Symbol'].isnull() == False)].GARD_ID))

2009

In [7]:
gard_symbol_hgnc = gard_gene_df[(gard_gene_df['Gene_Symbol'].isnull() == False) & 
             (gard_gene_df['HGNC'].isnull() == False)][['Gene_Symbol','HGNC']].drop_duplicates()

#Some dictionaries for convenient mapping later
sym2hgnc = {row['Gene_Symbol']:row['HGNC'] for i,row in gard_symbol_hgnc.iterrows()}
hgnc2sym = {row['HGNC']:row['Gene_Symbol'] for i,row in gard_symbol_hgnc.iterrows()}

In [8]:
#Use MyGene.info, part of biothings, to get NCBI id's
#So we can use NCBI go annotationsS
mg = mygene.MyGeneInfo()
mg_hgnc = mg.querymany([i for i in gard_symbol_hgnc['HGNC']], scopes='hgnc', fields='entrezgene', species='human')
hgnc2ncbi = {i.get('query') : i.get('entrezgene') for i in mg_hgnc if i.get('entrezgene') is not None}

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-2730...done.
Finished.
18 input query terms found no hit:
	['21340', '3777', '33445', '43724', '12599', '22433', '31376', '22993', '25855', '14260', '16706', '
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [9]:
len(hgnc2ncbi)

2712

In [10]:
no_hgnc_hits = gard_symbol_hgnc[(gard_symbol_hgnc['HGNC'].isin(hgnc2ncbi.keys())==False)]

In [11]:
mg_symbol = mg.querymany([i for i in no_hgnc_hits['Gene_Symbol']], scopes='symbol', fields='entrezgene', species='human')


querying 1-18...done.
Finished.
2 input query terms found no hit:
	['SPG23', 'FMR3']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [12]:
hgnc2ncbi.update({sym2hgnc.get(i.get('query')) : i.get('entrezgene') for i in mg_symbol if i.get('entrezgene') is not None})

In [13]:
len(hgnc2ncbi)

2728

In [14]:
sym2ncbi = {hgnc2sym[i]:hgnc2ncbi[i] for i in hgnc2ncbi.keys()}
sym_hits = sym2ncbi.keys()

In [15]:
#Search by symbol for those where HGNC returned nothing
mg_symbol = mg.querymany(
list(gard_gene_df[(gard_gene_df['Gene_Symbol'].isnull()==False) &
             (gard_gene_df['Gene_Symbol'].isin(sym_hits)==False)]['Gene_Symbol']), 
    scopes='symbol', fields='entrezgene', species='human')

querying 1-4...done.
Finished.
3 input query terms found no hit:
	['SPG23', 'FMR3', 'DHS6S1']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [16]:
sym2ncbi.update({i.get('query') : i.get('entrezgene') for i in mg_symbol if i.get('entrezgene') is not None})

In [17]:
ncbi2sym = {sym2ncbi.get(i):i for i in sym2ncbi.keys()}

In [18]:
#These three genes have not NCBI ID based on search via gene symbol or HGNC id
sym_hits = sym2ncbi.keys()

gard_gene_df[(gard_gene_df['Gene_Symbol'].isnull()==False) &
             (gard_gene_df['Gene_Symbol'].isin(sym_hits)==False)]

,GARD_ID,GARD_Disease,Disease_Gene_Association,Gene_Symbol,Gene_Name,Gene_Refs,OMIM,Ensembl,Genatlas,SwissProt,HGNC,Reactome,IUPHAR
288,336,Spastic paraplegia 23,Disease-causing germline mutation(s) in,SPG23,spastic paraplegia 23 (autosomal recessive),[HGNC:21340],NaN,NaN,NaN,NaN,21340,NaN,NaN
1647,2378,Fragile XE syndrome,Disease-causing germline mutation(s) in,FMR3,fragile X mental retardation associated 3,"[HGNC:3777, Genatlas:FMR3]",NaN,NaN,FMR3,NaN,3777,NaN,NaN
5418,9179,North Carolina macular dystrophy,Disease-causing germline mutation(s) in,DHS6S1,"DNase1 hypersensitivity, chromosome 6, site 1",[OMIM:616842],616842,NaN,NaN,NaN,NaN,NaN,NaN


### Gene Ontology and annotations

We obtain the gene onotlogy from the obolibrary, this has the the full GO dag.

Then we need annotations that link genes to GO terms. We explore two resources for this:

* The EBI curated gaf file, provided by geneonoloty.org
* The NCBI curate gene2go file, obtained using GOAtools

In [19]:
gene_ontology = from_file("http://purl.obolibrary.org/obo/go.owl")
gene_ontology.graph.nodes.get('GO:0003723')

/Users/sanjakjs/opt/anaconda3/envs/ontograph/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:286: SyntaxWarning: <Element '{http://purl.org/dc/elements/1.1/}description' at 0x7ff3c8a46180> contains text but no `xsd:datatype`
  meta.annotations.add(self._extract_literal_pv(child))
/Users/sanjakjs/opt/anaconda3/envs/ontograph/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:286: SyntaxWarning: <Element '{http://purl.org/dc/elements/1.1/}title' at 0x7ff3c8a46220> contains text but no `xsd:datatype`
  meta.annotations.add(self._extract_literal_pv(child))
/Users/sanjakjs/opt/anaconda3/envs/ontograph/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:286: SyntaxWarning: <Element '{http://www.w3.org/2002/07/owl#}versionInfo' at 0x7ff3c8a46400> contains text but no `xsd:datatype`
  meta.annotations.add(self._extract_literal_pv(child))
/Users/sanjakjs/opt/anaconda3/envs/ontograph/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:113: SyntaxWarning: unknown element in `owl:ObjectPr

{'identifier': 'GO:0003723',
 'label': 'RNA binding',
 'namespace': 'molecular_function'}

In [20]:
#Obtained from http://geneontology.org/gene-associations/goa_human.gaf.gz
gene_annotations = GafReader("goa_human.gaf")


HMS:0:00:12.268476 609,183 annotations READ: goa_human.gaf 


In [21]:
#From NCBI via goatools
fin_gene2go = download_ncbi_associations()

# Read NCBI's gene2go. Store annotations in a list of namedtuples
objanno = Gene2GoReader(fin_gene2go, taxids=[9606])

# Get namespace2association where:
#    namespace is:
#        BP: biological_process               
#        MF: molecular_function
#        CC: cellular_component
#    assocation is a dict:
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()

for nspc, id2gos in ns2assoc.items():
    print("{NS} {N:,} annotated mouse genes".format(NS=nspc, N=len(id2gos)))

  EXISTS: gene2go
HMS:0:00:04.967049 330,404 annotations, 20,688 genes, 18,642 GOs, 1 taxids READ: gene2go 
CC 19,433 annotated mouse genes
BP 18,501 annotated mouse genes
MF 18,194 annotated mouse genes


In [22]:
#NCBI ID to GO mappings from NCBI gene2go table
ncbi2go = {}
for assoc in objanno.associations:
    if assoc.DB_ID in ncbi2go:
        if assoc.NS in ncbi2go[assoc.DB_ID]:
            ncbi2go[assoc.DB_ID][assoc.NS].append({'GO_ID':assoc.GO_ID,'Qualifier':assoc.Qualifier})
        else:
            ncbi2go[assoc.DB_ID][assoc.NS] = [{'GO_ID':assoc.GO_ID,'Qualifier':assoc.Qualifier}]
    else:
        ncbi2go[assoc.DB_ID] = {assoc.NS:[{'GO_ID':assoc.GO_ID,'Qualifier':assoc.Qualifier}]}
    

In [23]:
#36 genes have no gene ontology annotation in NCBI
ncbi_no_go = set([int(i) for i in sym2ncbi.values()]).difference(set(ncbi2go.keys()))
len(ncbi_no_go)

36

In [24]:
#Gene symbol to GO mapppings from geneontology.org GOA table
gene2go = {}
for assoc in gene_annotations.get_associations():
    if assoc.DB_Symbol in gene2go:
        if assoc.NS in gene2go[assoc.DB_Symbol]:
            gene2go[assoc.DB_Symbol][assoc.NS].append({'GO_ID':assoc.GO_ID,'Qualifier':assoc.Qualifier})
        else:
            gene2go[assoc.DB_Symbol][assoc.NS] = [{'GO_ID':assoc.GO_ID,'Qualifier':assoc.Qualifier}]
    else:
        gene2go[assoc.DB_Symbol] = {assoc.NS:[{'GO_ID':assoc.GO_ID,'Qualifier':assoc.Qualifier}]}
        

### Disease to GO through GO annotations

In [25]:
#How many GARD disease gene symbols have at least one GO annotation?
gard_genes = set(gard_gene_df['Gene_Symbol'][(gard_gene_df['Gene_Symbol'].isnull()==False)])
gard_sym_go = gard_genes.intersection(set(gene2go.keys()))
gard_sym_nogo = gard_genes.difference(set(gene2go.keys()))

#Add additional annotations found through NCBI
gene2go.update({i:ncbi2go[int(sym2ncbi[i])] for i in gard_sym_nogo if i in sym2ncbi and int(sym2ncbi[i]) in ncbi2go})

In [26]:
#How many GARD disease gene symbols have at least one GO annotation?
gard_genes = set(gard_gene_df['Gene_Symbol'][(gard_gene_df['Gene_Symbol'].isnull()==False)])
gard_sym_go = gard_genes.intersection(set(gene2go.keys()))
gard_sym_nogo = gard_genes.difference(set(gene2go.keys()))

[len(gard_genes), len(gard_sym_go), len(gard_sym_nogo)]

[2732, 2696, 36]

This shows that the majority, 2696 of 2732, GARD disease associated genes have a GO annotation. What of the remaining 75 gene symbols? There are two obvious reasons for missing GO annotations: a) the gene is truly not annotated in GO by EBI and b) the EBI annotation uses a synonym gene symbol and therefore fails to map to our database. The former issue is not solvable in this context, but the latter may be.

#### Searching for Gene Symbol aliases

In [27]:
def gene_alias(hgnc_id):
    """
    This function queries genenames.org for know alias and old gene symbols associated
    with an HGNC ID code.

    Approach taken from https://www.genenames.org/help/rest/
    """
    
    headers = {
    'Accept': 'application/json',
    }

    uri = 'http://rest.genenames.org'
    path = '/fetch/hgnc_id/{0}'.format(hgnc_id)

    target = urllib.parse.urlparse(uri+path)
    method = 'GET'
    body = ''

    h = http.Http()

    response, content = h.request(
        target.geturl(),
        method,
        body,
        headers)

    if response['status'] == '200':
        # assume that content is a json reply
        # parse content with the json module 
        data = json.loads(content)
        if data['response']['numFound'] > 0:
            all_symbols = []
            for sym in ['symbol','prev_symbol','alias_symbol']:
                sym_alias = data['response']['docs'][0].get(sym)
                if sym_alias is not None:
                    if isinstance(sym_alias,list):
                        all_symbols.extend(sym_alias)
                    elif isinstance(sym_alias,str):
                        all_symbols.append(sym_alias)

            return all_symbols
        else:
            return None
    else:
        return None


In [28]:
symbol_hgnc = gard_gene_df[(gard_gene_df['Gene_Symbol'].isin(gard_sym_nogo)) & 
                           (gard_gene_df['HGNC'].isnull() == False)][['Gene_Symbol','HGNC']].drop_duplicates()

gard_sym_nogo_aliases = {}
for index, row in symbol_hgnc.iterrows():
    row_alias = gene_alias(row['HGNC'])
    if row_alias is not None:
        gard_sym_nogo_aliases[row['Gene_Symbol']] = row_alias

In [29]:
#We found info for 34/36 of the gene names without GO annotation
len(gard_sym_nogo_aliases.keys())

34

In [30]:
#All of these gene names have at least 3 names
Counter([len(i) for i in gard_sym_nogo_aliases])

Counter({9: 1, 5: 24, 6: 5, 3: 1, 4: 2, 7: 1})

In [31]:
gard_sym_alias_go = {}
for sym_nogo in gard_sym_nogo_aliases.keys():
    for alias_sym in gard_sym_nogo_aliases.get(sym_nogo):
        if alias_sym in gene2go:
            gard_sym_alias_go[sym_nogo] = alias_sym

In [32]:
#We found GO annotations through aliases for 1 out of the 36 gene name without direct annotation
len(gard_sym_alias_go)

1

In [33]:
gard_sym_alias_go

{'MT-TP': 'MTTP'}

### Update the gene to GO annotations via aliases

In [34]:
for gard_sym in gard_sym_alias_go.keys():
    gene2go[gard_sym] = gene2go[gard_sym_alias_go[gard_sym]]

In [35]:
#Update to how many GARD disease gene symbols have at least one GO annotation?
gard_sym_go = gard_genes.intersection(set(gene2go.keys()))
gard_sym_nogo = gard_genes.difference(set(gene2go.keys()))

[len(gard_genes), len(gard_sym_go), len(gard_sym_nogo)]

[2732, 2697, 35]

##### We still have 35 GARD gene symbols without GO Annotations. More could be mapped by accessing the Uniprot ID's from HGNC and finding GOA's that way. The remaining could be followed up with manually to see if there is an issue.

In [36]:
gard_gene_df[gard_gene_df['Gene_Symbol'].isin(gard_sym_nogo)]

,GARD_ID,GARD_Disease,Disease_Gene_Association,Gene_Symbol,Gene_Name,Gene_Refs,OMIM,Ensembl,Genatlas,SwissProt,HGNC,Reactome,IUPHAR
288,336,Spastic paraplegia 23,Disease-causing germline mutation(s) in,SPG23,spastic paraplegia 23 (autosomal recessive),[HGNC:21340],NaN,NaN,NaN,NaN,21340,NaN,NaN
663,848,Behçet disease,Major susceptibility factor in,IL12A-AS1,IL12A antisense RNA 1,"[HGNC:49094, Ensembl:ENSG00000244040]",NaN,ENSG00000244040,NaN,NaN,49094,NaN,NaN
1231,1709,"Deafness, isolated, due to mitochondrial trans...",Disease-causing germline mutation(s) in,MT-TH,mitochondrially encoded tRNA-His (CAU/C),"[OMIM:590040, Genatlas:TRNH, Ensembl:ENSG00000...",590040,ENSG00000210176,TRNH,NaN,7487,NaN,NaN
1232,1709,"Deafness, isolated, due to mitochondrial trans...",Disease-causing germline mutation(s) in,MT-TS1,mitochondrially encoded tRNA-Ser (UCN) 1,"[HGNC:7497, Ensembl:ENSG00000210151, OMIM:5900...",590080,ENSG00000210151,MT-TS1,NaN,7497,NaN,NaN
1340,1839,Transient neonatal diabetes mellitus,Disease-causing germline mutation(s) in,HYMAI,hydatidiform mole associated and imprinted,"[OMIM:606546, Ensembl:ENSG00000283122, HGNC:53...",606546,ENSG00000283122,HYMAI,NaN,5326,NaN,NaN
1634,2356,Follicular lymphoma,Part of a fusion gene in,IGH,immunoglobulin heavy locus,"[HGNC:5477, Genatlas:IGH@, SwissProt:Q6P089]",NaN,NaN,IGH@,Q6P089,5477,NaN,NaN
1647,2378,Fragile XE syndrome,Disease-causing germline mutation(s) in,FMR3,fragile X mental retardation associated 3,"[HGNC:3777, Genatlas:FMR3]",NaN,NaN,FMR3,NaN,3777,NaN,NaN
2003,3094,Keratoderma palmoplantar deafness,Disease-causing germline mutation(s) in,MT-TS1,mitochondrially encoded tRNA-Ser (UCN) 1,"[HGNC:7497, Ensembl:ENSG00000210151, OMIM:5900...",590080,ENSG00000210151,MT-TS1,NaN,7497,NaN,NaN
2332,3671,Mitochondrial DNA-associated Leigh syndrome,Disease-causing germline mutation(s) in,MT-TL1,mitochondrially encoded tRNA-Leu (UUA/G) 1,"[Genatlas:MT-TL1, Ensembl:ENSG00000209082, OMI...",590050,ENSG00000209082,MT-TL1,NaN,7490,NaN,NaN
2333,3671,Mitochondrial DNA-associated Leigh syndrome,Disease-causing germline mutation(s) in,MT-TK,mitochondrially encoded tRNA-Lys (AAA/G),"[OMIM:590060, Ensembl:ENSG00000210156, Genatla...",590060,ENSG00000210156,MT-TK,NaN,7489,NaN,NaN


## Example GO term similarity for two diseases

In [37]:
gard_gene_df[gard_gene_df['Gene_Symbol'].isin(gard_sym_go)].head()

,GARD_ID,GARD_Disease,Disease_Gene_Association,Gene_Symbol,Gene_Name,Gene_Refs,OMIM,Ensembl,Genatlas,SwissProt,HGNC,Reactome,IUPHAR
0,1,GRACILE syndrome,Disease-causing germline mutation(s) in,BCS1L,"BCS1 homolog, ubiquinol-cytochrome c reductase...","[OMIM:603647, Ensembl:ENSG00000074582, Genatla...",603647,ENSG00000074582,BCS1L,Q9Y276,1020,Q9Y276,NaN
1,3,Ablepharon macrostomia syndrome,Disease-causing germline mutation(s) in,TWIST2,twist family bHLH transcription factor 2,"[Ensembl:ENSG00000233608, OMIM:607556, Reactom...",607556,ENSG00000233608,TWIST2,Q8WVJ9,20670,Q8WVJ9,NaN
3,5,Abetalipoproteinemia,Disease-causing germline mutation(s) in,MTTP,microsomal triglyceride transfer protein,"[Ensembl:ENSG00000138823, Genatlas:MTTP, React...",157147,ENSG00000138823,MTTP,P55157,7467,P55157,NaN
5,7,Acromicric dysplasia,Disease-causing germline mutation(s) in,FBN1,fibrillin 1,"[Reactome:P35555, Genatlas:FBN1, OMIM:134797, ...",134797,ENSG00000166147,FBN1,P35555,3603,P35555,NaN
6,7,Acromicric dysplasia,Disease-causing germline mutation(s) in,LTBP3,latent transforming growth factor beta binding...,"[Reactome:Q9NS15, Genatlas:LTBP3, Ensembl:ENSG...",602090,ENSG00000168056,LTBP3,Q9NS15,6716,Q9NS15,NaN


In [38]:
go1 = gene2go['BCS1L']
go2 = gene2go['TWIST2']
print([go1['MF'],go2['MF']])

[[{'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005524', 'Qualifier': {'enables'}}], [{'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0046983', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0000977', 'Qualifier': {'enables'}}, {'GO_ID': 'GO:0000981', 'Qu

In [39]:
go1['MF']

[{'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005524', 'Qualifier': {'enables'}}]

In [40]:
go2['MF']

[{'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0005515', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0046983', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0000977', 'Qualifier': {'enables'}},
 {'GO_ID': 'GO:0000981', 'Qualifier': {'enables'}}]

In [41]:
similarity = gene_ontology.similarity('GO:0005515', 'GO:0046983', ic_metric="intrinsic_ic_sanchez")
# Access a single similarity metric
similarity.lin
# Access all similarity metrics
similarity.results()

{'node_0': 'GO:0005515',
 'node_1': 'GO:0046983',
 'node_0_subsumes_1': True,
 'node_1_subsumes_0': False,
 'n_common_ancestors': 3,
 'n_union_ancestors': 4,
 'batet': 0.75,
 'batet_log': 1.0,
 'ic_metric': 'intrinsic_ic_sanchez',
 'mica': 'GO:0005515',
 'resnik': 4.712053931186439,
 'resnik_scaled': 0.46118000993019104,
 'lin': 0.6488656392578207,
 'jiang': 0.16393801599303162,
 'jiang_seco': 0.7504319258465685}

# Disease - Phenotype

In [42]:
# Connect to disease ontology graph neo4j database
uri = "bolt://disease.ncats.io:80"
driver = GraphDatabase.driver(uri, auth=("neo4j", ""))

gard_phen_query = \
"""
MATCH r=(d:DATA)-[:PAYLOAD]->(n:S_GARD)-[:R_hasPhenotype]->(p:S_HP)<-[:PAYLOAD]-(z)
WHERE d.is_rare=true
RETURN DISTINCT d.id as `GARD_ID`, d.name as `GARD_Disease`,z.id as `HPO_ID`, z.label as `HPO_Phenotype`
ORDER BY d.id
"""

gard_phen_res = driver.session().run(gard_phen_query)

In [43]:
gard_phen_data = [dict(record) for record in gard_phen_res]
gard_phen_df = pd.DataFrame(gard_phen_data)

In [44]:
gard_phen_df

,GARD_ID,GARD_Disease,HPO_ID,HPO_Phenotype
0,1,GRACILE syndrome,HP:0100613,Death in early adulthood
1,1,GRACILE syndrome,HP:0012465,Elevated hepatic iron concentration
2,1,GRACILE syndrome,HP:0012464,Decreased transferrin saturation
3,1,GRACILE syndrome,HP:0003128,Lactic acidosis
4,1,GRACILE syndrome,HP:0003355,Aminoaciduria
...,...,...,...,...
93791,13818,Sphingosine phosphate lyase insufficiency synd...,HP:0000407,Sensorineural hearing impairment
93792,13818,Sphingosine phosphate lyase insufficiency synd...,HP:0000252,Microcephaly
93793,13818,Sphingosine phosphate lyase insufficiency synd...,HP:0001888,Lymphopenia
93794,13818,Sphingosine phosphate lyase insufficiency synd...,HP:0000007,Autosomal recessive inheritance


In [52]:
len(set(gard_phen_df.GARD_ID))

3578

In [45]:
phenotype_ontology  = from_file("http://purl.obolibrary.org/obo/hp.obo")


/Users/sanjakjs/opt/anaconda3/envs/ontograph/lib/python3.8/site-packages/nxontology/imports.py:63: UnicodeWarning: unsound encoding, assuming ISO-8859-1 (73% confidence)
  onto = Prontology(handle=handle)


In [46]:
similarity = phenotype_ontology.similarity('HP:0100613', 'HP:0003355', ic_metric="intrinsic_ic_sanchez")
# Access a single similarity metric
similarity.lin
# Access all similarity metrics
similarity.results()

{'node_0': 'HP:0100613',
 'node_1': 'HP:0003355',
 'node_0_subsumes_1': False,
 'node_1_subsumes_0': False,
 'n_common_ancestors': 1,
 'n_union_ancestors': 15,
 'batet': 0.06666666666666667,
 'batet_log': 0.02547695440020664,
 'ic_metric': 'intrinsic_ic_sanchez',
 'mica': 'HP:0000001',
 'resnik': 0.0,
 'resnik_scaled': 0.0,
 'lin': 0.0,
 'jiang': 0.05706133276381606,
 'jiang_seco': 0.11100666803057246}

## Output

In [48]:
gard_phen_df.to_csv("data/gard2hpo.csv",index=False)

In [49]:
gard_gene_df.to_csv("data/gard2gene.csv",index=False)

In [51]:
import pickle

with open('data/gene2go.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(gene2go, f, pickle.HIGHEST_PROTOCOL)